In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import re
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
df=pd.read_csv('data_preprocessing.csv')

In [6]:
df.drop(['startphrase'], axis=1)

,ending0,ending1,ending2,ending3,label
0,continues to talk as he begins pressing the gu...,continues playing the piano while the boy cont...,finishes playing the drum and pauses before th...,parts the flip and covers the equipment he made.,2
1,swimming in him and cheers him on.,makes several pictures and throws a ball.,grabs a mallet and throws the ball.,is falling near the pool in attempt.,2
2,reaches out and takes someone's hand.,is soaked through very bare curtains.,puts his leg around someone's neck.,is quiet - eyed with woman.,0
3,is soaked through very bare curtains.,puts his leg around someone's neck.,"stands, ready to usher someone out.",is quiet - eyed with woman.,2
4,are high and warm.,hangs on his wrists.,drops out of view.,is shaven and bald.,0
...,...,...,...,...,...
294,applies soap and shaving cream to her face.,is talking to the camera in the kitchen.,shows how to wrap a gift around present.,starts talking to the camera in the bathroom.,1
295,pulls a hole on the ground.,is pulled out with a knife.,cuts the string holding an apple.,is attached to a released point.,2
296,moves around and fourth.,hops to the side.,laying down looking down.,proceeds to smile slightly.,3
297,finish watching the other woman perform.,spins the hoop with her hand.,lifts the weight and spins around.,"begins riding a heavy, elliptical bike.",1


In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

def get_likelihood(prompt, ending):
    input_text = prompt + ending
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss.item()
    return loss

def select_best_ending(row):
    prompt = "Some context here to complete the sentences: "  
    endings = [row['ending0'], row['ending1'], row['ending2'], row['ending3']]
    losses = [get_likelihood(prompt, ending) for ending in endings]
    best_ending_idx = losses.index(min(losses))
    return best_ending_idx


/home/youwei2/miniconda3/envs/ds/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
df['best_ending'] = df.apply(select_best_ending, axis=1)

In [9]:
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending
0,The man plays continuously around on the drum ...,continues to talk as he begins pressing the gu...,continues playing the piano while the boy cont...,finishes playing the drum and pauses before th...,parts the flip and covers the equipment he made.,2,1
1,Another man on the field catches the ball. The...,swimming in him and cheers him on.,makes several pictures and throws a ball.,grabs a mallet and throws the ball.,is falling near the pool in attempt.,2,2
2,"Someone sits across from him, looking small an...",reaches out and takes someone's hand.,is soaked through very bare curtains.,puts his leg around someone's neck.,is quiet - eyed with woman.,0,0
3,"Someone sits across from him, looking small an...",is soaked through very bare curtains.,puts his leg around someone's neck.,"stands, ready to usher someone out.",is quiet - eyed with woman.,2,1
4,Someone's gaze is steady. His high white shirt...,are high and warm.,hangs on his wrists.,drops out of view.,is shaven and bald.,0,3
...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,applies soap and shaving cream to her face.,is talking to the camera in the kitchen.,shows how to wrap a gift around present.,starts talking to the camera in the bathroom.,1,3
295,An arrow goes through the hole of a donut. An ...,pulls a hole on the ground.,is pulled out with a knife.,cuts the string holding an apple.,is attached to a released point.,2,1
296,Another child moves around behind her. More ch...,moves around and fourth.,hops to the side.,laying down looking down.,proceeds to smile slightly.,3,1
297,The woman exercise on front the ocean spinning...,finish watching the other woman perform.,spins the hoop with her hand.,lifts the weight and spins around.,"begins riding a heavy, elliptical bike.",1,2


In [11]:
def calculate_surprisal(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        logits = outputs.logits

    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
    surprisal = -torch.sum(log_probs * torch.nn.functional.one_hot(input_ids, num_classes=logits.size(-1)).float())
    
    return surprisal.item()

In [12]:
columns=['ending0', 'ending1', 'ending2', 'ending3']

for index,row in df.iterrows():
    for col in columns:
        sentence = row[col]
        surprisal = calculate_surprisal(sentence)
        df.at[index,col] = surprisal
df

,startphrase,ending0,ending1,ending2,ending3,label,best_ending
0,The man plays continuously around on the drum ...,106.062126,105.51252,96.884735,90.222755,2,1
1,Another man on the field catches the ball. The...,86.585938,79.253586,99.089836,70.123375,2,2
2,"Someone sits across from him, looking small an...",85.449265,55.319748,85.888588,61.42556,0,0
3,"Someone sits across from him, looking small an...",55.319748,85.888588,87.065315,61.42556,2,1
4,Someone's gaze is steady. His high white shirt...,42.692604,53.533142,48.238132,56.046124,0,3
...,...,...,...,...,...,...,...
294,A woman is standing inside her kitchen. She,86.396469,75.682159,77.956757,84.997215,1,3
295,An arrow goes through the hole of a donut. An ...,74.971596,63.508228,62.587616,57.772198,2,1
296,Another child moves around behind her. More ch...,50.764687,44.882004,57.364052,64.445938,3,1
297,The woman exercise on front the ocean spinning...,70.271095,72.533035,68.238945,108.430801,1,2


In [13]:
#human vs GPT: label vs best_ending
comparison = df['label'] == df['best_ending']
correct_predictions = comparison.sum()
accuracy = correct_predictions / len(df)

In [14]:
accuracy

0.3411371237458194